In [ ]:
# Run this cell to download dependencies
%pip install lingo_api==19.0.6
%pip install pandas
%pip install plotly
%pip install geopy

In [6]:
import lingo_api as lingo
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from geopy import distance

In [7]:
# TSP Callback
# state = 1: The LINGO model has just added new subtours to the cuts
# state = 2: The model has been solved
# state = 0: Working on solving the submodel or finding new cuts
#
#
def cbSolver(pEnv, nReserved, uData):
    state   = np.array([-1.0], dtype=np.double)
    errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Model_State"], state)
    
    if state[0] == 1:

        newTours   = np.array([-1.0], dtype=np.double)
        totalTours = np.array([-1.0], dtype=np.double)
        kurTime = np.array([-1.0], dtype=np.double)
        
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["New_Tours"], newTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Total_Tours"], totalTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Time"], kurTime)
        
        print(f"Total Cuts: {int(totalTours[0]):-5} \t New Cuts: {int(newTours[0]):-5} \t Time Used: {kurTime[0]:-5.4f}")
    
    elif state[0] == 2 and uData["Solve_Printed"]==False:
        uData["Solve_Printed"] = True
          
        solved     = np.array([-1.0], dtype=np.double)
        newTours   = np.array([-1.0], dtype=np.double)
        totalTours = np.array([-1.0], dtype=np.double)
        kurTime    = np.array([-1.0], dtype=np.double)
        
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["SOLVED"], solved)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["New_Tours"], newTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Total_Tours"], totalTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Time"], kurTime)

        if solved[0] == 0:
            printStr = f"TSP Solved in {kurTime[0]:.4f} seconds"
        else:
            printStr = f"TSP was unable to be solved before running out of cuts"
        print("\n=======================================================================================")
        print(printStr)

    else:
        pass
    return 0

def cbError(pEnv, uData, nErrorCode, errorText):
    # A exception will be displayed if this callback is called
    raise(nErrorCode, errorText)
    


## LINGO - Python TSP

This TSP model is implemented using Dantzig, Fulkerson, Johnson (DFJ) subtour elimination constraints with the maximum number of cuts each iteration controlled by the user. The data for this model comes from US_Tour.csv, however the data can be swaped out with another data set with the columns; city, latitude, longitude. The LINGO model is included in the same directory as this jupyter notebook.

There are seven LINGO pointers that this model uses to send and recive data that must be set/allocated in Python.

- **Pointer1**: *MAXTOUR* a model parameter that sets the maximum number of subtours that can be added to the model.
- **Pointer2**: *MAXADD* a model parameter that sets the maximum number of subtours that can be added at each iteration.
- **Pointer3**: *CITY* a model set passed as a NumPy array of city names that will be visited on the tour.
- **Pointer4**: *LATI* a model parameter passed as a NumPy array of latitudes of each city on the tour.
- **Pointer5**: *LNGT* a model parameter passed as a NumPy array of longitudes of each city on the tour.
- **Pointer6**: *Y* a model variabel matrix passed as a NumPy array of size numberOfCities*numberOfCities where $Y_{i,j} = 1$ if path form city $i$ to city $j$ is used $0$ otherwise.
- **Pointer7**: *DIST* a model parameter for of distance between node $i$ and $j$. To allocate memmory for this return data create an empty NumPy array of size numberOfCities*numberOfCities.
- **Pointer8**: *SOLVED* a model variable that is is $0$ if the TSP was solved $1$ otherwise.

In [71]:
# Make swapping datasets painless by describing them as a dictionary 
Huston = {
            "CSV":"Houston_Texas_Neighborhoods_Lat_Long_List.csv",
            "Map_Center":{'lon': -95.36, 'lat': 29.75},
            "Map_ZOOM":9,
}

US_Tour = {
            "CSV":"US_Tour.csv",
            "Map_Center":{'lon': -98.35, 'lat': 39.50},
            "Map_ZOOM":2,
}

In [77]:
#======= User Supplied Data

DATA_SET = US_Tour    # DATA_SET a dictionary that describes a dataset
MAXTOUR = 250         # Maximum tours that can be added to the model
MAXADD  = 50          # Maximum tours that can added at each iteration

#===========

# csv data
df = pd.read_csv(DATA_SET["CSV"])
# model 
lngFile = "tspLoop.lng"

# data for pointres
CITY = df["city"].values
LATI = df["latitude"].values  
LNGT = df["longitude"].values

        

numCities = len(CITY)

Y = np.empty(numCities*numCities)
DIST = np.empty(numCities*numCities)
SOLVED = np.array([-1.0])

# Callback Data
uData =  {"Prefix":"Lingo API",
          "Model_State":np.array(["STATE"],dtype="|S102"),
          "New_Tours":np.array(["ADDED"],dtype="|S102"),
          "Total_Tours":np.array(["TOTALTOURS"],dtype="|S102"),
          "Time":np.array(["KURTIME"],dtype="|S102"),
          "SOLVED":np.array(["SOLVED"],dtype="|S102"),
          "Solve_Printed":False,
}


In [78]:
# Create a model object
model = lingo.Model(lngFile)

# set all pointers in the order that they appear in tspLoop.lng
model.set_pointer("Pointer1",MAXTOUR,lingo.PARAM)
model.set_pointer("Pointer2",MAXADD,lingo.PARAM)
model.set_pointer("Pointer3",CITY,lingo.SET)
model.set_pointer("Pointer4",LATI,lingo.PARAM)
model.set_pointer("Pointer5",LNGT,lingo.PARAM)
model.set_pointer("Pointer6",Y,lingo.VAR)
model.set_pointer("Pointer7",DIST,lingo.PARAM)
model.set_pointer("Pointer8",SOLVED,lingo.VAR)

# set all callback functions and user data
model.set_cbSolver(cbSolver)
model.set_uData(uData)

# solve the model
lingo.solve(model)

# Fix Data
Y = np.reshape(Y,(numCities,numCities))
DIST = np.reshape(DIST,(numCities,numCities))

Total Cuts:    17 	 New Cuts:    17 	 Time Used: 0.1270
Total Cuts:    31 	 New Cuts:    14 	 Time Used: 0.2450
Total Cuts:    38 	 New Cuts:     7 	 Time Used: 0.3750
Total Cuts:    39 	 New Cuts:     1 	 Time Used: 0.5000

TSP Solved in 0.5000 seconds


In [79]:
# Orginaize data to be ploted
j_map = np.zeros(numCities, dtype=np.int32)


pos = 0
for ij, val in np.ndenumerate(Y):
    if val >= 0.5:
        i = ij[0]
        j = ij[1]
        j_map[pos] = j
        pos += 1 
        
entry_pos = 0
lon_sorted = np.zeros(numCities)
lat_sorted = np.zeros(numCities)
city_sorted = np.empty(numCities, dtype=object)
sorted_indx = np.zeros(numCities,dtype=int)

lon_sorted[entry_pos] = LNGT[0]
lat_sorted[entry_pos] = LATI[0]
city_sorted[0] = CITY[0]
sorted_indx[0] = 0


for i in range(1,len(j_map)-1):
    lon_sorted[i] = LNGT[j_map[entry_pos]]
    lat_sorted[i] = LATI[j_map[entry_pos]]
    city_sorted[i] = CITY[j_map[entry_pos]]
    sorted_indx[i] = j_map[entry_pos]
    entry_pos = j_map[entry_pos]
    
lon_sorted[-1] = LNGT[0]
lat_sorted[-1] = LATI[0]
city_sorted[-1] = CITY[0]
sorted_indx[-1] = 0

In [80]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lon_sorted,
    lat = lat_sorted,
    text = city_sorted,
    marker = {'size': 10, 'color' : 'red'}))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': DATA_SET["Map_Center"],
        'style': "stamen-terrain",
        'zoom': DATA_SET["Map_ZOOM"]})


fig.show()

In [76]:
header = ['City', '', 'total Dist']
ijdist = np.empty((len(city_sorted)))
accDist = np.empty((len(city_sorted)))

ijdist[0] = 0
accDist[0] = 0

for k in range(1, len(sorted_indx)):
    i = sorted_indx[k-1]
    j = sorted_indx[k]
    ijdist[k] = DIST[i,j]
    accDist[k] = accDist[k-1] + DIST[i,j]


tspDF = pd.DataFrame({'City':city_sorted, 'dist':ijdist, 'total Dist':accDist})
tspDF

,City,dist,total Dist
0,Acres Home,0.000000,0.000000
1,Greater Inwood,4.268316,4.268316
2,Willowbrook,11.846117,16.114433
3,Carverdale,10.803096,26.917529
4,Fairbanks,2.111071,29.028600
...,...,...,...
83,Lake Houston,18.798390,360.435842
84,Kingwood,9.726207,370.162049
85,IAH Airport,16.101135,386.263184
86,Greater Greenspoint,9.248832,395.512016
